<a href="https://colab.research.google.com/github/leonrafael29/W266_Final_Project/blob/main/mBART/Shuffled_MBart_Single_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount google drive to use for file saving and loading

In [ ]:
from google.colab import files, drive
drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive

Mounted at /content/gdrive/
/content/gdrive/MyDrive


Install requirements

In [ ]:
!pip install sentencepiece -q
!pip install transformers -q
!pip install datasets -q


# !wget -N https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip . -q
# !unzip -q -n BLEURT-20.zip

#Downloads the 3-layer distilled model, which is much smaller.
# !wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D3.zip .
# !unzip BLEURT-20-D3.zip

     |████████████████████████████████| 1.3 MB 41.4 MB/s 
     |████████████████████████████████| 5.5 MB 32.1 MB/s 
     |████████████████████████████████| 182 kB 79.3 MB/s 
     |████████████████████████████████| 7.6 MB 62.6 MB/s 
     |████████████████████████████████| 451 kB 31.5 MB/s 
     |████████████████████████████████| 212 kB 94.0 MB/s 
     |████████████████████████████████| 115 kB 83.5 MB/s 
     |████████████████████████████████| 127 kB 89.1 MB/s 


Imports

In [ ]:
import csv
import numpy as np
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, \
    MBart50TokenizerFast, MBartConfig,\
    TrainingArguments, Trainer
    

Global variables

In [ ]:
ORIGINAL_MODEL_CHECKPOINT = 'facebook/mbart-large-50-many-to-many-mmt'
MODEL_CHECKPOINT = 'Mbart/Model/Tiny/checkpoint-1000'

PAIRS = [
    'en-zh',
    'zh-en',
    'en-es',
    'es-zh',
    'es-en',
    'zh-es',
    ]
MBART_DATA = {
    'en-zh': {
        'size': 69020,
        'train': 48444,
        'val': 10381,
        'src': 'en',
        'tgt': 'zh',
        'src_tkn': 'en_XX',
        'tgt_tkn':'zh_CN',
        'tkn': 'zh_CN',
        'reverse': False,
        'train_path':f'Mbart/Data/fixed_train_pairs.csv',
        'val_path':f'Mbart/Data/fixed_val_pairs.csv',
        'test_path':f'Mbart/Data/fixed_test_pairs.csv',
        },
    'zh-en': {
        'size': 69020,
        'train': 48444,
        'val': 10381,
        'src': 'zh',
        'tgt': 'en',
        'src_tkn': 'zh_CN',
        'tgt_tkn':'en_XX',
        'tkn': 'en_XX',
        'reverse': True,
        'train_path':f'Mbart/Data/fixed_train_pairs.csv',
        'val_path':f'Mbart/Data/fixed_val_pairs.csv',
        'test_path':f'Mbart/Data/fixed_test_pairs.csv',
        },
    'en-es': {
        'size': 238511,
        'train': 167210,
        'val': 35831,
        'src': 'en',
        'tgt': 'es',
        'src_tkn': 'en_XX',
        'tgt_tkn':'es_XX',
        'tkn': 'es_XX',
        'reverse': False,
        'train_path':f'Mbart/Data/fixed_train_pairs.csv',
        'val_path':f'Mbart/Data/fixed_val_pairs.csv',
        'test_path':f'Mbart/Data/fixed_test_pairs.csv',
        },
    'es-zh': {
        'size': 65408,
        'train': 45796,
        'val': 9814,
        'src': 'es',
        'tgt': 'zh',
        'src_tkn': 'es_XX',
        'tgt_tkn':'zh_CN',
        'tkn': 'zh_CN',
        'reverse': False,
        'train_path':f'Mbart/Data/fixed_train_pairs.csv',
        'val_path':f'Mbart/Data/fixed_val_pairs.csv',
        'test_path':f'Mbart/Data/fixed_test_pairs.csv',
        },
    'es-en': {
        'size': 238511,
        'train': 167210,
        'val': 35831,
        'src': 'es',
        'tgt': 'en',
        'src_tkn': 'es_XX',
        'tgt_tkn':'en_XX',
        'tkn': 'en_XX',
        'reverse': True,
        'train_path':f'Mbart/Data/fixed_train_pairs.csv',
        'val_path':f'Mbart/Data/fixed_val_pairs.csv',
        'test_path':f'Mbart/Data/fixed_test_pairs.csv',
        },
    'zh-es': {
        'size': 65408,
        'train': 45796,
        'val': 9814,
        'src': 'zh',
        'tgt': 'es',
        'src_tkn': 'zh_CN',
        'tgt_tkn':'es_XX',
        'tkn': 'es_XX',
        'reverse': True,
        'train_path':f'Mbart/Data/fixed_train_pairs.csv',
        'val_path':f'Mbart/Data/fixed_val_pairs.csv',
        'test_path':f'Mbart/Data/fixed_test_pairs.csv',
        },
    }

DATASET = 'news_commentary'
MAX_LENGTH = 50
MAX_NEW_TOKENS = 50
TRUNCATION = True
PADDING = True
RETURN_TENSORS = 'pt'
BLEURT_CHECKPOINT = './BLEURT-20-D3'
N_EXAMPLES = 100

%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512


Load Model, Metrics and Tokenizer

In [ ]:
config = MBartConfig.from_pretrained(ORIGINAL_MODEL_CHECKPOINT)

config.encoder_layers = 1
config.decoder_layers = 1
config.num_hidden_layers = 1

In [ ]:
# Load the metrics model and tokenizer for use in the next cell

model = MBartForConditionalGeneration.from_pretrained(ORIGINAL_MODEL_CHECKPOINT, config=config)
tokenizer = MBart50TokenizerFast.from_pretrained(ORIGINAL_MODEL_CHECKPOINT)
#model = torch.load('Mbart/Model/epoch-1')

Downloading:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mbart-large-50-many-to-many-mmt were not used when initializing MBartForConditionalGeneration: ['model.decoder.layers.7.self_attn.out_proj.weight', 'model.encoder.layers.7.final_layer_norm.weight', 'model.decoder.layers.1.fc2.bias', 'model.encoder.layers.4.self_attn.v_proj.bias', 'model.decoder.layers.1.self_attn.v_proj.weight', 'model.encoder.layers.4.self_attn.q_proj.bias', 'model.encoder.layers.4.final_layer_norm.weight', 'model.decoder.layers.2.final_layer_norm.weight', 'model.encoder.layers.10.self_attn.k_proj.weight', 'model.decoder.layers.1.encoder_attn_layer_norm.bias', 'model.decoder.layers.6.fc1.weight', 'model.encoder.layers.8.self_attn.v_proj.weight', 'model.decoder.layers.6.self_attn.v_proj.weight', 'model.encoder.layers.9.final_layer_norm.weight', 'model.encoder.layers.11.fc2.weight', 'model.encoder.layers.2.self_attn_layer_norm.bias', 'model.decoder.layers.4.fc1.bias', 'model.decoder.layers.4.encoder_attn_layer_norm.weight

Downloading:   0%|          | 0.00/529 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
def supervised_preprocessor(src_data, tgt_data, tokenizer, src_tkn, tgt_tkn):
  tokenizer.src_lang = src_tkn
  tokenizer.tgt_lang = tgt_tkn
 
  inputs = tokenizer(
        text=[np.array2string(src_data)],
        text_target=[np.array2string(tgt_data)],
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=TRUNCATION,
        return_tensors=RETURN_TENSORS,
        )

  return {'input_ids':inputs.input_ids[0],
           'attention_mask':inputs.attention_mask[0],
           'labels':inputs.labels[0],
          }

In [ ]:
 class TranslationDataIterator:
    
    def __init__(self,
                 tokenizer,
                 n_examples,
                 max_load_at_once,
                 data_filename,
                  max_length=MAX_LENGTH,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.max_load_at_once = max_load_at_once
        self.data_filename = data_filename
        self.max_length = max_length
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()

        # Load first chunk of max_load_at_once examples
        self.df_curr_loaded = self._load_next_chunk(0)
        self.curr_idx_in_load = 0
    
    def _load_next_chunk(self, idx):
        load_start = idx
        load_end = idx + self.max_load_at_once

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this chunk
        load_idx_skip = self.row_order[:load_start] + self.row_order[load_end:]
        self.df_curr_loaded = pd.read_csv(self.data_filename, skiprows=load_idx_skip)
    
    def __len__(self):
        return self.n_examples
    
    def __getitem__(self, idx):
        if self.df_curr_loaded is None or self.curr_idx_in_load >= len(self.df_curr_loaded):
            self._load_next_chunk(idx)
            self.curr_idx_in_load = 0
        
        src_tkn = self.df_curr_loaded[['src_tkn']].values.astype(str)[self.curr_idx_in_load][0]
        tgt_tkn = self.df_curr_loaded[['tgt_tkn']].values.astype(str)[self.curr_idx_in_load][0]
        src_data = self.df_curr_loaded[['src']].values.astype(str)[self.curr_idx_in_load]
        tgt_data = self.df_curr_loaded[['tgt']].values.astype(str)[self.curr_idx_in_load]

        self.curr_idx_in_load += 1

        item_data = supervised_preprocessor(
            src_data,
            tgt_data,
            self.tokenizer,
            src_tkn,
            tgt_tkn,
        )
        
        return item_data
    
    def __call__(self):
        for i in range(self.__len__()):
            yield self.__getitem__(i)
            
            if i == self.__len__()-1:
                self.on_epoch_end()
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [ ]:
MODEL_PATH=f"Mbart/Model/Shuffled"
BATCH_SIZE=32

TRAINER_PARAMS = TrainingArguments(
    MODEL_PATH,
    evaluation_strategy='no',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=1,
    dataloader_drop_last=True,
    resume_from_checkpoint=True,

)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Create the data generators for train and validation data, tensorflow version

max_length = MAX_LENGTH
max_load_at_once = 200 

# Load individual dataset, score BLEU and BLEURT scores for dataset
train_file = 'Mbart/Data/fixed_train_pairs.csv'

print("Training everything")

train_data_generator = TranslationDataIterator(
  tokenizer=tokenizer,
  n_examples=400000,
  max_load_at_once=max_load_at_once,
  data_filename=train_file,
  max_length=max_length,
)

mbart_trainer = Trainer(
  model=model,
  args=TRAINER_PARAMS,
  train_dataset=train_data_generator,
  eval_dataset=None,
  compute_metrics=None,
)
mbart_trainer.train()
torch.save(model, 'Mbart/Model/Shuffled/Shuffled-400k')
    

Training everything


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 400000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 12500
  Number of trainable parameters = 287557632


Step,Training Loss
500,4.859200
1000,3.167200
1500,2.852400
2000,2.590000
2500,2.945700
3000,4.347800
3500,3.955100
4000,3.399900


Saving model checkpoint to Mbart/Model/Shuffled/checkpoint-500
Configuration saved in Mbart/Model/Shuffled/checkpoint-500/config.json
Model weights saved in Mbart/Model/Shuffled/checkpoint-500/pytorch_model.bin
Saving model checkpoint to Mbart/Model/Shuffled/checkpoint-1000
Configuration saved in Mbart/Model/Shuffled/checkpoint-1000/config.json
Model weights saved in Mbart/Model/Shuffled/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to Mbart/Model/Shuffled/checkpoint-1500
Configuration saved in Mbart/Model/Shuffled/checkpoint-1500/config.json
Model weights saved in Mbart/Model/Shuffled/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to Mbart/Model/Shuffled/checkpoint-2000
Configuration saved in Mbart/Model/Shuffled/checkpoint-2000/config.json
Model weights saved in Mbart/Model/Shuffled/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to Mbart/Model/Shuffled/checkpoint-2500
Configuration saved in Mbart/Model/Shuffled/checkpoint-2500/config.json
Model weigh

In [ ]:
torch.save(model, 'Mbart/Model/Single/epoch-1')

In [ ]:
# torch.save(model.state_dict(), 'Mbart/Model/Tiny/epoch-2-state')

In [ ]:
!ls

sample_data


AttributeError: ignored

In [ ]:
troubleshooting_data_generator[0]

{'inputs': {'input_ids': tensor([[250025,    378,     25,   1189,  37415,   5714,      4,  26871,  55899,
            17989,  60353, 226934,    264,  48616, 123843,   1988,     30, 124920,
             1535,  37928,    213,   4617,   2920,   3300,  13447,     37,   5250,
             2100,    213, 156455,   1420,  12590,  23470,   7447,   1493,  95539,
                4,   1288,  27839,   1635,   1288,  48616,   4491,  27043,   5525,
            11608,  53989,  32870,  72926,      2]], device='cuda:0'),
  'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
           1, 1]], device='cuda:0'),
  'decoder_input_ids': tensor(250005)},
 'labels': tensor([[250005,    378,     25,    284,  26897,      8,    576, 110245,      7,
               4,    576,  28214,  26850,      7,   6805,   1005, 190037,      7,
             121,   1011,    377,  40578,     10,     21

In [ ]:
tokenizer.decode(troubleshooting_data_generator[0]['inputs']['input_ids'])

TypeError: ignored

In [ ]:
tokenizer.decode(troubleshooting_data_generator[0]['labels'])

In [ ]:
troubleshooting_data_generator[0]

In [ ]:
original = tokenizer.decode(troubleshooting_data_generator[0]['inputs']['input_ids'][0])
original

In [ ]:
generated_none = model.generate(input_ids=troubleshooting_data_generator[0]['inputs']['input_ids'], max_length=50)
generated_none

In [ ]:
tokenizer.decode(generated_none[0])

In [ ]:
generated_zh = model.generate(input_ids=troubleshooting_data_generator[0]['inputs']['input_ids'], forced_bos_token_id=tokenizer.lang_code_to_id['zh_CN'], max_length=50)
generated_zh

In [ ]:
tokenizer.decode(generated_zh[0])

In [ ]:
generated_es = model.generate(input_ids=troubleshooting_data_generator[0]['inputs']['input_ids'], forced_bos_token_id=tokenizer.lang_code_to_id['es_XX'], max_length=50)
generated_es

In [ ]:
tokenizer.decode(generated_es[0])

In [ ]:
model.config

In [ ]:
src_tkn

In [ ]:
    mbart_trainer.train()

In [ ]:
print(torch.cuda.memory_summary())

In [ ]:
model.train()

In [ ]:
print(torch.cuda.memory_summary())

In [ ]:
 torch.cuda.synchronize()
 torch.cuda.empty_cache()